In [1]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, MetaData
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import SQLDatabase
from llama_index.core.objects import SQLTableNodeMapping
from llama_index.core.objects import ObjectIndex
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import SQLTableSchema


# Importar a secret key
load_dotenv()
api_key = os.getenv("secret_key")

# Iniciar o db
url = 'manutencao_industrial.db'           
engine = create_engine(f'sqlite:///{url}') 

# Objeto para coletar as informações da eng
metadata_obj = MetaData()  
metadata_obj.reflect(engine)
print(metadata_obj.tables.keys())

# Iniciando o modelo
modelo="llama-3.3-70b-versatile"
modelo_hf_emb="BAAI/bge-m3"
#Settings.llm = Groq(model=modelo, api_key = api_key)
llm = Groq(model = modelo, api_key = api_key)
Settings.embed_model = HuggingFaceEmbedding(model_name = modelo_hf_emb)


# Criando um mapeamento do db - SQLTableNodeMapping
sql_database = SQLDatabase(engine)
table_node_map = SQLTableNodeMapping(sql_database)

c:\Users\camil\OneDrive\Documentos\Projetos\2025-06 LLM com acesso ao db\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dict_keys(['equipamentos', 'ordem_tecnico', 'ordens_manutencao', 'tecnicos'])


In [29]:
from typing import List
from llama_index.core.query_pipeline import FnComponent
from llama_index.core.objects import SQLTableSchema

# Criar as tabelas como objeto para o Llama conseguir identificar
table_squema_objs = []
for nome_tabela in metadata_obj.tables.keys():
    schema_info = sql_database.get_single_table_info(nome_tabela)
    # Use o schema como descrição (pode adicionar mais, se quiser)
    table_squema_objs.append(
    SQLTableSchema(table_name=nome_tabela)
)
  

# Criando o retriever
obj_index = ObjectIndex.from_objects(table_squema_objs, table_node_map, VectorStoreIndex)
obj_retriever = obj_index.as_retriever(similarity_top_k=1)


# Criando a descrição das tabelas
def descricao_tabela(schema_tabelas: List[SQLTableSchema]):
    descricao_str = []
    for tabela_schema in schema_tabelas:
        # Usa apenas o que está em context_str, que já foi salvo no retriever
        descricao = tabela_schema.context_str or sql_database.get_single_table_info(tabela_schema.table_name)
        descricao_str.append(descricao)
    return '\n\n'.join(descricao_str)

contexto_tabela = FnComponent(fn=descricao_tabela)

In [30]:
# Prompt 1 - adaptado para PT

from llama_index.core import PromptTemplate

texto2sql = """Dada uma pergunta em linguagem natural, crie uma consulta {dialect} sintaticamente correta para executar e, em seguida, verifique os resultados da consulta e retorne a resposta. Você pode ordenar os resultados por uma coluna relevante para retornar os exemplos mais informativos no banco de dados.

Nunca consulte todas as colunas de uma tabela específica. Pergunte apenas por algumas colunas relevantes, de acordo com a pergunta.

Preste atenção para usar apenas os nomes de colunas que você pode ver na descrição do esquema. Tenha cuidado para não consultar colunas que não existem. Preste atenção em qual coluna está em qual tabela. Além disso, qualifique os nomes das colunas com o nome da tabela quando necessário.

Use o seguinte formato, cada um em uma linha:

Pergunta: Pergunta aqui
ConsultaSQL: Consulta SQL para executar
ResultadoSQL: Resultado da ConsultaSQL
Resposta: Resposta final aqui

Use apenas as tabelas listadas abaixo.

{schema}

Pergunta: {pergunta_user}
ConsultaSQL:
"""

prompt_1_template = PromptTemplate(texto2sql, dialect = engine.dialect.name)

In [31]:
def prompt_1_fn(pergunta_user: str, schema: str, conversation_history: str = "") -> str:
    schema_com_hist = schema + "\nContexto anterior:\n" + conversation_history
    # Aqui você chama o .format() no template
    return prompt_1_template.format(pergunta_user=pergunta_user, schema=schema_com_hist)

prompt_1_comp = FnComponent(fn=prompt_1_fn)

In [32]:
# Função para a consulta_sql

from llama_index.core.llms import ChatResponse

def resposta_sql (resposta: ChatResponse) -> str:
    '''O dado de entrada será ChatResponse
    -> str informa que a saída deverá ser do tipo string'''
    conteudo_resposta = resposta.message.content
    
    sql_consulta = conteudo_resposta.split("ConsultaSQL: ", 1)[-1].split("ResultadoSQL: ", 1)[0]
    return sql_consulta.strip().strip('```').strip()
# Strip para tirar textos em branco.
# strip('```') para remover markdowns que possam vir no código da llm

consulta_sql = FnComponent(fn=resposta_sql)

In [33]:
# Função para o resultado_sql

from llama_index.core.retrievers import SQLRetriever

resultado_sql = SQLRetriever(sql_database)

In [34]:
# Função para o prompt 2:

prompt_2_str = '''
    Você é o "Assitente de consulta de banco de dados da SiderTech Solutions".
    Dada a seguinte pergunta, a consulta SQL correspondente e o resultado SQL, responda à pergunta de modo agradável e objetivamente.
    Evite iniciar conversas com cumprimentos e apresentações, como "Olá".

    Pergunta: {pergunta_user}
    Consulta SQL: {consulta}
    Resultado SQL: {resultado}
    Resposta:
    '''

prompt_2 = PromptTemplate(
    prompt_2_str,
)

In [35]:
def prompt_2_fn(pergunta_user: str, consulta: str, resultado: str, conversation_history: str = "") -> str:
    texto_completo = prompt_2.format(
        pergunta_user=pergunta_user,
        consulta=consulta,
        resultado=resultado
    )
    # Você pode acrescentar conversation_history se quiser usar contexto ali também
    return texto_completo

prompt_2_comp = FnComponent(fn=prompt_2_fn)


In [36]:
# Função para criar o contexto de meória

from llama_index.core.query_pipeline import FnComponent

conversation_history = []

def memory_component(input_text: str):
    conversation_history.append(input_text)
    # retorna todo o histórico concatenado (ou formatado)
    return "\n".join(conversation_history)

memory = FnComponent(fn=memory_component)

In [37]:
from llama_index.core.query_pipeline import QueryPipeline, InputComponent

qp = QueryPipeline(
    modules = {                               
        'entrada': InputComponent(), 
        'memory': memory,         
        'acesso_tabela': obj_retriever,   
        'contexto_tabela': contexto_tabela,   
        'prompt_1': prompt_1_comp,     # wrapper função aqui
        'llm_1': llm,
        'consulta_sql': consulta_sql,
        'resultado_sql': resultado_sql,
        'prompt_2': prompt_2_comp,     # wrapper função aqui
        'llm_2': llm,
    },
    verbose=False
)

C:\Users\camil\AppData\Local\Temp\ipykernel_17548\1762823326.py:3: DeprecationWarning: Call to deprecated class QueryPipeline. (QueryPipeline has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

It is recommended to switch to the Workflows API for a more flexible and powerful experience.

See the docs for more information workflows: https://docs.llamaindex.ai/en/stable/understanding/workflows/)
  qp = QueryPipeline(


In [38]:
qp.add_chain(['entrada', 'memory', 'acesso_tabela', 'contexto_tabela'])
qp.add_link('memory', 'prompt_1', dest_key='conversation_history')
qp.add_link('memory', 'prompt_2', dest_key='conversation_history')
qp.add_link('entrada', 'prompt_1', dest_key='pergunta_user')
qp.add_link('contexto_tabela', 'prompt_1', dest_key='schema')
qp.add_chain(['prompt_1', 'llm_1', 'consulta_sql', 'resultado_sql'])
qp.add_link('entrada', 'prompt_2', dest_key='pergunta_user')
qp.add_link('consulta_sql', 'prompt_2', dest_key='consulta')
qp.add_link('resultado_sql', 'prompt_2', dest_key='resultado')
qp.add_link('prompt_2', 'llm_2')

In [39]:
print(str(qp.run(query='Qual técnico trabalhou na ordem 32')))

assistant: O técnico que trabalhou na ordem 32 foi o "Tecnico 3".


In [40]:
print(str(qp.run(query='Qual a especialidade dele?')))

assistant: A especialidade dele é elétrica.


In [41]:
print(str(qp.run(query='Quais os equipamentos que tiveram manutenção nos últimos 3 meses?')))

assistant: Os equipamentos que tiveram manutenção nos últimos 3 meses são:

- Equipamento 9: 25/05/2025
- Equipamento 2: 22/05/2025 e 22/04/2025
- Equipamento 3: 07/06/2025, 04/06/2025 e 10/05/2025
- Equipamento 8: 18/04/2025, 13/04/2025 e 31/03/2025
- Equipamento 1: 17/05/2025 e 31/03/2025
- Equipamento 4: 02/05/2025 e 02/04/2025
- Equipamento 10: 23/05/2025, 18/05/2025, 03/05/2025 e 23/04/2025
- Equipamento 5: 27/05/2025, 03/05/2025, 14/04/2025 e 11/06/2025
- Equipamento 6: 20/06/2025 e 06/06/2025
- Equipamento 7: 21/06/2025


In [42]:
print(str(qp.run(query='Qual o nome do técnico que trabalhou em mais ordens de manutenção?')))

assistant: O nome do técnico que trabalhou em mais ordens de manutenção é Tecnico 3.
